In [1]:
from six.moves import cPickle
import os
import matplotlib.pyplot as plt
import numpy as np
import importlib
import utils as ut
import functions as fu 
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
importlib.reload(ut)
X_train, y_train, Y_train = ut.loadData('data_batch_1', clipping=True)
X_val, y_val, Y_val = ut.loadData('data_batch_2', clipping=True)
X_test, y_test, Y_test = ut.loadData('data_batch_3', clipping=True)

## normalize with mean and std of train set 
mean_X = np.mean(X_train, axis=1)
std_X = np.std(X_train, axis=1)

X_train -= np.outer(mean_X, np.ones(X_train.shape[1]))
X_train /= np.outer(std_X, np.ones(X_train.shape[1]))

X_val -= np.outer(mean_X, np.ones(X_val.shape[1]))
X_val /= np.outer(std_X, np.ones(X_val.shape[1]))

X_test -= np.outer(mean_X, np.ones(X_test.shape[1]))
X_test /= np.outer(std_X, np.ones(X_test.shape[1]))

# Experiments

In [ ]:
importlib.reload(ut)

GDparams = [{"lambda":0, "n_epochs":40, "n_batch":100, "eta":.1}, {"lambda":0, "n_epochs":40, "n_batch":100, "eta":.001}, 
          {"lambda":.1, "n_epochs":40, "n_batch":100, "eta":.001}, {"lambda":1, "n_epochs":40, "n_batch":100, "eta":.001}]

np.random.seed(42)
seeds = np.random.randint(0, 100, 5)
stats = {i:{"val_loss":[], "train_loss":[], "val_acc":[], "train_acc":[], "test_acc":[]}for i in range(4)}

for i, GDparam in enumerate(GDparams): 
    for seed in seeds:
        np.random.seed(seed)
        W = np.random.normal(mu, sigma, (K,d))
        b = np.random.normal(mu, sigma, (K,1))
        W, b, train_loss, val_loss, train_acc, val_acc = ut.minibatchGD(X_train, Y_train, y_train,  X_val, Y_val, y_val, GDparam, W, b, verbose=False)

        stats[i]["train_loss"].append(train_loss[-1])
        stats[i]["val_loss"].append(val_loss[-1])
        stats[i]["train_acc"].append(train_acc[-1])
        stats[i]["val_acc"].append(val_acc[-1])
        stats[i]["test_acc"].append(ut.ComputeAccuracy(X_test, y_test, W, b))
    
    ut.montage(W, GDparam)
    ut.plot_metric(train_loss, val_loss, GDparam, type="loss")
    ut.plot_metric(train_acc, val_acc, GDparam, type="accuracy")
np.save("History/stats.npy", stats)